# Chapter 4. Module gotchas

# Import subtleties


## Module needs to be explicitly imported somewhere
Simply having the `sec1.py` file in subpackage directory is not enough.

In [1]:
%cd ..

/Users/babkin/reseng


In [2]:
from lib import chap4
chap4.sec1

Hello, lib!
Hello, chap4!


AttributeError: module 'lib.chap4' has no attribute 'sec1'

How to access `sec1` module?
- add `from . import sec1` to `chap4/__init__.py`
- `from lib.chap4 import sec1`
- `import lib.chap4.sec1`

Notice that `from lib import chap3.sec3` is not allowed.

In [3]:
from lib.chap4 import sec1
sec1

Hello, chap4, sec1.
Hello, chap4, sec2.


<module 'lib.chap4.sec1' from '/Users/babkin/reseng/lib/chap4/sec1.py'>

Now that `sec1` is loaded, we can also refer to it as `chap4.sec1`:

In [4]:
chap4.sec1

<module 'lib.chap4.sec1' from '/Users/babkin/reseng/lib/chap4/sec1.py'>

## Have you noticed "sec2"?

Even though `sec2` is not imported in `chap4/__init__.py`, and neither did we explicitly import it in this notebook, it is still available.

This is because it is loaded in `sec1.py`:  
`from . import sec2`

In [5]:
chap4.sec2

<module 'lib.chap4.sec2' from '/Users/babkin/reseng/lib/chap4/sec2.py'>

In [6]:
sec1.sec2

<module 'lib.chap4.sec2' from '/Users/babkin/reseng/lib/chap4/sec2.py'>

In [7]:
chap4.sec1.sec2

<module 'lib.chap4.sec2' from '/Users/babkin/reseng/lib/chap4/sec2.py'>

## Loading happens only once per module
If we now execute additional import statements for `sec2`, it will be available under new names, but all these names would refer to the same entity.

Module also would not be reloaded (does not say "hello").

If you need to reload module without restarting kernel:
- `importlib.reload()`
- `%autoreload`

In [8]:
import lib.chap4.sec2
lib.chap4.sec2

<module 'lib.chap4.sec2' from '/Users/babkin/reseng/lib/chap4/sec2.py'>

In [9]:
from lib.chap4 import sec2
sec2

<module 'lib.chap4.sec2' from '/Users/babkin/reseng/lib/chap4/sec2.py'>

In [10]:
import lib.chap4.sec2 as s2
s2

<module 'lib.chap4.sec2' from '/Users/babkin/reseng/lib/chap4/sec2.py'>

# Shared state

All variables defined in modules can be mutated or reassigned. Not just variables, but even function and Class definitions, since they all are just fields in module's `__dict__`.

This flexilibity can be useful, but can also be dangerous.

**With great power comes great responsibility.**

*Restart kernel*

In [1]:
%cd ..

/Users/babkin/reseng


In [2]:
from lib.chap4 import sec1
from lib.chap4 import sec2

Hello, lib!
Hello, chap4!
Hello, chap4, sec1.
Hello, chap4, sec2.


In [3]:
sec2.main()

My name is Section 2


In [4]:
sec2.my_name = 'Section 2, better than ever!'
sec2.main()

My name is Section 2, better than ever!


In [5]:
# possibly unexpected side-effect
sec1.mutate_sec2('Здесь был Вася Ж)')
sec2.main()

My name is Здесь был Вася Ж)


In [6]:
# functions are not special
sec2.main = lambda: print('Told you!')
sec2.main()

Told you!


## Suggestions
- Document your code.
  - use decriptive names
  - docstrings
  - \# comments
  - notebook markdown
- Use pseudo-private variables, `_underscored`.
  - don't show up as module, class or object members on tab completion
  - serve as additional documentation: "don't touch me"
  - are still not private and can be modified
- Use alternatives to your taste and situation.
  - classes
  - objects
  
Related Stack Overflow discussions:
- [Global state in Python module](https://stackoverflow.com/questions/17346428/global-state-in-python-module)
- [Maintain state without classes](https://stackoverflow.com/questions/11866419/how-to-maintain-state-in-python-without-classes)
- [Module as singleton](https://stackoverflow.com/questions/6255050/python-thinking-of-a-module-and-its-variables-as-a-singleton-clean-approach)
- [Globals and singletons](https://stackoverflow.com/questions/12066180/globals-and-singletons-in-python)

### Class properties are shared

If you *want* to share state between parts of you program, you can also use class and class properties.

In [7]:
class Box():
    weight = 10
    dimensions = [1, 2, 3]
# ...
print(Box.weight) # 10
print(Box.dimensions) # [1, 2, 3]
# ...
Box.weight = 5
Box.dimensions[2] = 1
# ...
print(Box.weight) # 5
print(Box.dimensions) # [1, 2, 1]

10
[1, 2, 3]
5
[1, 2, 1]


### Object properties are not shared
If you want to use entity in multiple places and maintain different internal state.

In [8]:
class Bag():
    def __init__(self, size):
        self.size = size
small_bag = Bag(1)
large_bag = Bag(2)

print(small_bag.size) # 1
print(large_bag.size) # 2

1
2
